In [7]:
import numpy as np
from scipy.stats import norm
from py_vollib.black_scholes import black_scholes as bs 
from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, rho, theta

# Black Scholes Formula

$C = \Phi(d_1)S_t - \Phi(d_2)Ke^{-rt}$

$d_1 = \frac{ln(\frac{S_t}{K}) + (r + \frac{\sigma^2}{2})t)}{\sigma \sqrt{t}}$

$d_2 = d_1 - \sigma \sqrt{t}$

$\frac{\partial d_1}{\partial \sigma} = \frac{rt-ln(\frac{S_t}{K})}{\sqrt{t}\sigma^2} + \frac{\sqrt{t}}{2}$

In [10]:
# black scholes model from model.ipynb

def black_scholes(r, S, K, T, sigma, type):

    d1 = (np.log(S/K) + (r+ sigma**2/2)*T)/(sigma*np.sqrt(T))

    d2 = d1 - sigma*np.sqrt(T)

    if type == "CALL":
        price = S*norm.cdf(d1,0,1) - K*np.exp(-r*T)*norm.cdf(d2,0,1)
    elif type == "PUT":
        price = K*np.exp(-r*T)*norm.cdf(-d2,0,1) - S*norm.cdf(-d1, 0, 1)

    return price


In [14]:
# confirm our model

r = 0.01    #risk free rate
S = 30 
K = 40
T = 240/365
sigma = 0.3
type = "CALL"
print("our call option price: " + str(black_scholes(r, S, K, T, sigma, type)))
print("correct call option price: " + str(bs("c", S, K, T, r, sigma)))


type = "PUT"
print("our put option price: " + str(black_scholes(r, S, K, T, sigma, type)))
print("correct put option price: " + str(bs("p", S, K, T, r, sigma)))

our call option price: 0.5132843798399405
their call option price: 0.5132843798399411
our put option price: 10.251133491653508
their put option price: 10.2511334916535


# Delta
Delta measures the rate of change of our option value $V$ with respect to the underliying asset: $\Delta = \frac{\partial V}{\partial S}$



$\Delta_{Call}= \frac{\partial (\Phi(d_1)S - \Phi(d_2)Ke^{-rt})}{\partial S}$

$= \Phi (d_1) + S \phi(d_1) \frac{\partial d_1}{\partial S} - e^{-RT}K\phi(d_2) \frac{\partial d_2}{\partial S}$

$= \Phi(d_1) + S \phi(d_1)\frac{\partial d_1}{\partial S} - e^{-rT}K\phi(d_2) \frac{\partial d_2}{\partial S}$

$= \Phi(d_1) + S \phi(d_1) \frac{1}{S \sigma \sqrt{T}} - e^{-rT}K\phi(d_2) \frac{1}{S \sigma \sqrt{T}}$

$= \Phi(d_1) + \frac{\phi (d_1)}{\sigma \sqrt{T}}(1 - e^{(d_1^2 - d_2^2)/2} \frac{K}{S e^{rT}})$

$= \Phi(d_1) + \frac{\phi (d_1)}{\sigma \sqrt{T}}(1 - \frac{S e^{rT}}{K} \frac{K}{S e^{rT}})$

$= \Phi(d_1)$




Put-Call-Parity:

$C + Ke^{-rT} = S + P$ 

$\Leftrightarrow P = C  + Ke^{-rT} - S$

$\Leftrightarrow \frac{\partial P}{\partial S} = \frac{\partial ( C  + Ke^{-rT} - S)}{\partial S}$

$\Leftrightarrow \Delta_{Put} = \Delta_{Call} - 1 = \Phi(d_1) - 1 = - \Phi(-d_1)$



In [22]:
def delta_black_scholes(r, S, K, T, sigma, type):

    d1 = (np.log(S/K) + (r+ sigma**2/2)*T)/(sigma*np.sqrt(T))

    if type == "CALL":
        price = norm.cdf(d1,0,1)
    elif type == "PUT":
        price = -norm.cdf(-d1,0,1)

    return price


type = "CALL"
print("our call option delta: " + str(delta_black_scholes(r, S, K, T, sigma, type)))
print("correct call option delta: " + str(delta("c", S, K, T, r, sigma)))


type = "PUT"
print("our put option delta: " + str(delta_black_scholes(r, S, K, T, sigma, type)))
print("correct put option delta: " + str(delta("p", S, K, T, r, sigma)))

our call option delta: 0.15058613984880015
correct call option delta: 0.15058613984880015
our put option delta: -0.8494138601511998
correct put option delta: -0.8494138601511998


# Gamma

Describes the rate of change between an option's delta $\Delta$ and the underlying asset price. High Gamma indicates a high change in Delta if the price of the underlying asset changes. $\Gamma = \frac{\partial \Delta}{\partial S}$


$\Gamma_{Call} = \frac{\partial (\Phi(d_1))}{\partial S}$

$= \phi(d_1) \cdot \frac{\partial d_1}{\partial S}$

$=  \frac{\phi(d_1)}{S \sigma \sqrt{T}}$

$\Gamma_{Put} = \frac{\partial ( - \Phi( - d_1))}{\partial S}$

$= -\phi(d_1) \cdot -\frac{\partial d_1}{\partial S}$

$=  \frac{\phi(d_1)}{S \sigma \sqrt{T}}$

In [25]:
def gamma_black_scholes(r, S, K, T, sigma, type):

    d1 = (np.log(S/K) + (r+ sigma**2/2)*T)/(sigma*np.sqrt(T))

    if type == "CALL":
        price = norm.pdf(d1,0,1)/(S * sigma * np.sqrt(T))
    elif type == "PUT":
        price = norm.pdf(d1,0,1)/(S * sigma * np.sqrt(T))

    return price


type = "CALL"
print("our call option gamma: " + str(gamma_black_scholes(r, S, K, T, sigma, type)))
print("correct call option gamma: " + str(gamma("c", S, K, T, r, sigma)))


type = "PUT"
print("our put option gamma: " + str(gamma_black_scholes(r, S, K, T, sigma, type)))
print("correct put option gamma: " + str(gamma("p", S, K, T, r, sigma)))

our call option gamma: 0.03203161102008452
correct call option gamma: 0.03203161102008452
our put option gamma: 0.03203161102008452
correct put option gamma: 0.03203161102008452


# Vega

Descirbes the rat of change between the options value and the volatility. Thus $\Lambda = \frac{\partial V}{\partial \sigma}$

$\Lambda_{Call}= \frac{\partial (\Phi(d_1)S - \Phi(d_2)Ke^{-rt})}{\partial \sigma}$


$= S\phi(d_1) \frac{\partial d_1}{\partial \sigma} - Ke^{-rt} \phi(d_2) \frac{\partial d_2}{\partial \sigma}$

$\frac{\partial d_1}{\partial \sigma} = \frac{rt-ln(\frac{S_t}{K})}{\sqrt{t}\sigma^2} + \frac{\sqrt{t}}{2}$

$= S\phi(d_1) \frac{\partial d_1}{\partial \sigma} - Ke^{-rt} \phi(d_2) \frac{\partial d_2}{\partial \sigma}$

SyntaxError: invalid syntax (3583367249.py, line 1)